<h1>Aim of Project: Building a database for Crimes report</h1>
<p>The goal of this project is to create a database named <b>crimes_db</b> with a table – <b>boston_crimes</b> – with appropriate <b>datatypes</b> for storing the data from the boston.csv file. We will be creating the table inside a schema named <b>crimes</b>. We will also create the <b>readonly</b> and <b>readwrite</b> groups with the appropriate privileges as we have discussed in this course. Finally, we will also need to create one user for each of these groups.</p>
<p>so for this below are the description what actually will be created</p>
<ul>
<li>Server: Postgres Server</li>
<li>Database: crimes.db</li>
<li>Schema: crime_db.crimes</li>
<li>Groups: readonly, readwrite</li>
<li>user to readonly group: data_analyst</li>
<li>user to readwrite group: data_scientist</li>
<li>Table: boston_crimes</li>
</ul>

<h3>Creating the crime Database</h3>

In [49]:
import psycopg2

conn = psycopg2.connect(dbname='dq', user='dq')
cur = conn.cursor()
conn.autocommit = True
cur.execute("CREATE DATABASE crime_db;")
conn.autocommit = False
conn.close()

ProgrammingError: database "crime_db" already exists


In [56]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")
conn.commit()
conn.close()

ProgrammingError: schema "crimes" already exists


<h3>Obtaining the column names and samples</h3>

In [4]:
import csv

with open('boston.csv','r') as file:
    reader = csv.reader(file)
    csv_data = []
    for row in reader:
        csv_data.append(row)
    col_headers = csv_data[0]
    first_row = csv_data[1]

In [5]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


<h3>Creating a function to examine column values</h3>

In [6]:
def get_col_value_set(csv_filename, col_index):
    with open(csv_filename,'r') as file:
        next(file)
        reader = csv.reader(file)
        col_val = set()
        for row in reader:
            col_val.add(row[col_index])
    return col_val

In [7]:
second_col = len(get_col_value_set('boston.csv', 1))
print(second_col)

219


In [8]:
for val in range(7):
    print(col_headers[val], " = ",len(get_col_value_set('boston.csv', val)))

incident_number  =  298329
offense_code  =  219
description  =  239
date  =  1177
day_of_the_week  =  7
lat  =  18177
long  =  18177


In [9]:
fourth_col = get_col_value_set('boston.csv', 4)
print(fourth_col)

{'Monday', 'Thursday', 'Sunday', 'Friday', 'Tuesday', 'Wednesday', 'Saturday'}


<h3>Finding the maximum length of column values</h3>

In [10]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [11]:
max_day_of_the_week = max(get_col_value_set('boston.csv',4))
print(max_day_of_the_week)

Wednesday


In [12]:
print(len(max_day_of_the_week))

9


In [13]:
max_description = max(get_col_value_set('boston.csv',2))
print(max_description)

WEAPON - OTHER - OTHER VIOLATION


In [14]:
print(len(max_description))

32


<h3>Creating the table:</h3>

In [15]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [16]:
print(first_row)

['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [17]:
with open('boston.csv','r') as file:
    reader = csv.reader(file)
    csv_data = []
    for row in reader:
        csv_data.append(row)
    for val in range(1,6):
        print(csv_data[val])

['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']
['2', '1402', 'VANDALISM', '2018-08-21', 'Tuesday', '42.30682138', '-71.06030035']
['3', '3410', 'TOWED MOTOR VEHICLE', '2018-09-03', 'Monday', '42.34658879', '-71.07242943']
['4', '3114', 'INVESTIGATE PROPERTY', '2018-09-03', 'Monday', '42.33418175', '-71.07866441']
['5', '3114', 'INVESTIGATE PROPERTY', '2018-09-03', 'Monday', '42.27536542', '-71.09036101']


<p>By looking at value of the various column, we can keep the data types as follows:
<ul>
<li>incident_number - Integer</li>
<li>offense_code - Integer</li>
<li>description - string(varchar)</li>
<li>date - date</li>
<li>day_of_the_week - enum</li>
<li>lat - decimal</li>
<li>long - decimal</li>
</ul>
</p>
<p>we keep the day_of_the_week as enum since its values are 7 only and fixed one</p>

In [20]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
#cur.execute("DROP TYPE weekdays CASCADE")
cur.execute("""
            CREATE TYPE weekdays as ENUM
            ('Monday','Tuesday','Wednesday',
            'Thursday','Friday','Saturday','Sunday');
            """)

In [21]:
conn.commit()
conn.close()

In [92]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute("DROP TABLE crimes.boston_crimes;")
cur.execute('''
            CREATE TABLE crimes.boston_crimes(
            incident_number INTEGER PRIMARY KEY,
            offense_code INTEGER,
            description varchar(200),
            date DATE,
            day_of_the_week weekdays,
            lat decimal,
            long decimal);
            ''')

<h3>Loading the data</h3>

In [93]:
with open('boston.csv') as file:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;",file)
    cur.execute("SELECT * FROM crimes.boston_crimes LIMIT 5;")

In [94]:
data = cur.fetchall()
for val in data:
    print(val)

(1, 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', Decimal('42.35779134'), Decimal('-71.13937053'))
(2, 1402, 'VANDALISM', datetime.date(2018, 8, 21), 'Tuesday', Decimal('42.30682138'), Decimal('-71.06030035'))
(3, 3410, 'TOWED MOTOR VEHICLE', datetime.date(2018, 9, 3), 'Monday', Decimal('42.34658879'), Decimal('-71.07242943'))
(4, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', Decimal('42.33418175'), Decimal('-71.07866441'))
(5, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', Decimal('42.27536542'), Decimal('-71.09036101'))


<h3>Revoking Public Privileges</h3>

In [95]:
cur.execute('''
        REVOKE ALL ON SCHEMA public FROM public;
''')

In [96]:
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

<h3>Creating User Group</h3>

In [89]:
cur.execute("CREATE GROUP readonly NOLOGIN;")

ProgrammingError: role "readonly" already exists


In [52]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")

In [97]:
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")

In [98]:
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")

In [99]:
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")

In [100]:
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

In [101]:
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

In [102]:
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

<h3>Creating Users</h3>

In [103]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")

In [104]:
cur.execute("GRANT data_analyst TO readonly;")

In [105]:
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")

In [106]:
cur.execute("GRANT data_scientist TO readwrite;")

In [107]:
cur.execute('''
SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readwrite';
''')

In [108]:
cur.fetchall()

[('readwrite', 'INSERT'),
 ('readwrite', 'SELECT'),
 ('readwrite', 'UPDATE'),
 ('readwrite', 'DELETE')]

In [109]:
cur.execute('''
SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readonly';
''')

In [110]:
cur.fetchall()

[('readonly', 'SELECT')]

In [111]:
conn.commit()
conn.close()